In [22]:
import pandas as pd
import keras
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = 9,6

from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings(action='ignore')

from generate_custome_models import generate_model1, generate_model2, generate_model3
from generate_pretrained_models import generate_resnet_model, generate_vgg_model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

# Data loading and preparation

In [23]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [24]:
BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES=10

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [25]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

# Fitting the custom deep learning models

In [26]:
LR=1e-3 
EPOCHS=100  
PATIENCE=10
MIN_DELTA=0.01
early_stop = EarlyStopping(monitor='val_accuracy', patience=PATIENCE, min_delta=MIN_DELTA)

model1=generate_model1(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model2=generate_model2(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model3=generate_model3(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [21]:
import io
from contextlib import redirect_stdout

def capture_model_summary(model):
    with io.StringIO() as buf, redirect_stdout(buf):
        model.summary()
        return buf.getvalue()

summary1 = capture_model_summary(model1)
summary2 = capture_model_summary(model2)
summary3 = capture_model_summary(model3)

lines1 = summary1.splitlines()
lines2 = summary2.splitlines()
lines3 = summary3.splitlines()
max_length = max(len(lines1), len(lines2), len(lines3))

lines1 += [''] * (max_length - len(lines1))
lines2 += [''] * (max_length - len(lines2))
lines3 += [''] * (max_length - len(lines3))

for line1, line2, line3 in zip(lines1, lines2, lines3):
    print(f"{line1:<80} {line2:<80} {line3:<80}")

Model: "sequential_9"                                                            Model: "sequential_10"                                                           Model: "sequential_11"                                                          
_________________________________________________________________                _________________________________________________________________                _________________________________________________________________               
 Layer (type)                Output Shape              Param #                    Layer (type)                Output Shape              Param #                    Layer (type)                Output Shape              Param #                  
=================================================================                =================================================================                =================================================================               
 conv2d_36 (Conv2D)         

# Fitting pre-trained arch

In [32]:
resnet_model=generate_resnet_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
vgg_model=generate_vgg_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [33]:
model1.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_15 (Flatten)        (None, 2048)              0         
                                                                 
 dense_40 (Dense)            (None, 10)                20490     
                                                                 
Total params: 23608202 (90.06 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [34]:
model2.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_16 (Flatten)        (None, 512)               0         
                                                                 
 dense_41 (Dense)            (None, 10)                5130      
                                                                 
Total params: 14719818 (56.15 MB)
Trainable params: 5130 (20.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
